In [2]:
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import pylab as pl
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import tabulate
from operator import add

from shapely import wkt
from shapely.geometry import shape, LineString, Point
    
# funciones desarrolladas
from functions.agrupar_dfs_censo import *
from functions.cargar_data import *
from functions.impresion import *

In [46]:
dd = pd.read_csv('tablas/dd_localidades.csv')
print(dd.shape[0])
dd.head()

8791


,loc_origen,loc_destino,personas_mig,pobl_origen,pobl_destino,cod,distancia_m
0,1020,2220,635,1304729,40657,10202220,583715
1,3221,2220,5,71258,40657,32212220,570608
2,3320,2220,10,19865,40657,33202220,547145
3,3321,2220,1,20524,40657,33212220,573837
4,3322,2220,1,25947,40657,33222220,583158


In [42]:
locs = cargar_datos_geo()[2][['CODLOC', 'geometry']]
locs.head()

/home/guillermo/anaconda3/envs/tesis/lib/python3.9/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


,CODLOC,geometry
0,2220,POINT (549273.219 6633915.063)
1,2521,POINT (442454.671 6652273.541)
2,2522,POINT (457921.001 6633361.660)
3,2621,POINT (468458.270 6601361.867)
4,2721,POINT (502880.928 6648162.053)


In [43]:
dd = dd.merge(locs, left_on='loc_origen', right_on='CODLOC')
dd = dd.merge(locs, left_on='loc_destino', right_on='CODLOC')
dd.drop(['CODLOC_x', 'CODLOC_y'], axis=1, inplace=True)
dd.rename({'geometry_x':'geom_ori', 'geometry_y':'geom_des'}, axis=1, inplace=True)
dd.head()

,loc_origen,loc_destino,personas_mig,pobl_origen,pobl_destino,cod,distancia_m,geom_ori,geom_des
0,1020,2220,635,1304729,40657,10202220,583715,POINT (573082.248 6145008.434),POINT (549273.219 6633915.063)
1,3221,2220,5,71258,40657,32212220,570608,POINT (571815.138 6157813.988),POINT (549273.219 6633915.063)
2,3320,2220,10,19865,40657,33202220,547145,POINT (565955.641 6179409.202),POINT (549273.219 6633915.063)
3,3321,2220,1,20524,40657,33212220,573837,POINT (570666.047 6153926.678),POINT (549273.219 6633915.063)
4,3322,2220,1,25947,40657,33222220,583158,POINT (594619.163 6157118.215),POINT (549273.219 6633915.063)


In [44]:
dd.to_csv('capas/loc_lines.csv', index=False)